In [1]:
import pandas as pd
import smile_extract

path = '/Users/raeed/Library/CloudStorage/OneDrive-UniversityofPittsburgh/0-projects/cst-rtt/cst-data/preTD/Dwight/2025-01-07/Dwight_2025-01-07_reward-type_medium_cst-rtt-dco_sorted.mat'
smile_data = smile_extract.direct_load_smile_data(path)

In [2]:
tf = smile_extract.compose_session_frame(smile_data)
tf

channel                          monkey            date_time block task  \
signal                                                                    
trial_id time                                                             
1        0 days 00:00:00         Dwight  2025-01-07 13:10:14        RTT   
         0 days 00:00:00.010000  Dwight  2025-01-07 13:10:14        RTT   
         0 days 00:00:00.020000  Dwight  2025-01-07 13:10:14        RTT   
         0 days 00:00:00.030000  Dwight  2025-01-07 13:10:14        RTT   
         0 days 00:00:00.040000  Dwight  2025-01-07 13:10:14        RTT   
...                                 ...                  ...   ...  ...   
152      0 days 00:00:04.340000  Dwight  2025-01-07 13:20:42        CST   
         0 days 00:00:04.350000  Dwight  2025-01-07 13:20:42        CST   
         0 days 00:00:04.360000  Dwight  2025-01-07 13:20:42        CST   
         0 days 00:00:04.370000  Dwight  2025-01-07 13:20:42        CST   
         0 days 00:00:04.380000  Dwight  2025-01-07 13:20:42        CST   

channel                           result            state hand position  \
signal                                                                x   
trial_id time                                                             
1        0 days 00:00:00         success  Reach to Center     61.511639   
         0 days 00:00:00.010000  success  Reach to Center     61.523815   
         0 days 00:00:00.020000  success  Reach to Center     61.523992   
         0 days 00:00:00.030000  success  Reach to Center     61.528734   
         0 days 00:00:00.040000  success  Reach to Center     61.521756   
...                                  ...              ...           ...   
152      0 days 00:00:04.340000  success          Success    131.019076   
         0 days 00:00:04.350000  success          Success    130.678476   
         0 days 00:00:04.360000  success          Success    130.328765   
         0 days 00:00:04.370000  success          Success    129.983302   
         0 days 00:00:04.380000  success          Success    129.675428   

channel                                                motor cortex  ...  \
signal                                    y          z        ch6u1  ...   
trial_id time                                                        ...   
1        0 days 00:00:00         796.000308  45.378573            0  ...   
         0 days 00:00:00.010000  796.008314  45.383986            0  ...   
         0 days 00:00:00.020000  796.014579  45.373014            0  ...   
         0 days 00:00:00.030000  796.005679  45.383058            0  ...   
         0 days 00:00:00.040000  795.999943  45.400846            1  ...   
...                                     ...        ...          ...  ...   
152      0 days 00:00:04.340000  899.375467  11.113030            0  ...   
         0 days 00:00:04.350000  899.449719  11.031885            0  ...   
         0 days 00:00:04.360000  899.540967  10.946598            0  ...   
         0 days 00:00:04.370000  899.620873  10.865358            0  ...   
         0 days 00:00:04.380000  899.682388  10.795263            1  ...   

channel                                                                  \
signal                          ch122u2 ch122u3 ch122u4 ch123u1 ch123u2   
trial_id time                                                             
1        0 days 00:00:00              0       0       0       0       0   
         0 days 00:00:00.010000       0       0       0       0       0   
         0 days 00:00:00.020000       1       0       0       0       0   
         0 days 00:00:00.030000       0       0       0       0       0   
         0 days 00:00:00.040000       0       0       0       0       0   
...                                 ...     ...     ...     ...     ...   
152      0 days 00:00:04.340000       0       0       0       0       1   
         0 days 00:00:04.350000       0       0       0       0       0   
        

In [56]:
import numpy as np
spike_times = smile_extract.get_smile_spike_times(smile_data)
recording_duration = (
    spike_times
    .groupby('trial_id')
    ['timestamp']
    .agg(np.ptp)
    .sum()
)
min_firing_rate = 10
average_firing_rate = (
    spike_times
    .groupby(['channel','unit'])
    ['timestamp']
    .agg('count')
    .apply(lambda x: x / recording_duration.total_seconds())
)
# (
#     spike_times
#     .reset_index()
#     .set_index(['channel','unit'])
#     .loc[average_firing_rate > min_firing_rate]
#     .reset_index()
#     .set_index(['trial_id','snippet_id'])
# )
num_units = len(average_firing_rate)
num_units_removed = (average_firing_rate <= min_firing_rate).sum()
print(f"Removing {num_units_removed} of {num_units} units with average firing rate less than {min_firing_rate} Hz.")

Removing 49 of 138 units with average firing rate less than 10 Hz.


In [ ]:
import altair as alt

alt.data_transformers.enable(max_rows=10000)
raster = (
    alt.Chart(
        tf
        .loc[152,'motor cortex']
        .stack()
        .rename('spikes')
        .loc[lambda x: x > 0]
        .reset_index()
        .assign(time=lambda x: x['time'].dt.total_seconds())
    ).mark_tick().encode(
        x='time:Q',
        y=alt.Y('signal:N').axis(labels=False,ticks=False).title('Neuron'),
    )
    .configure_tick(
        bandSize=2,
        thickness=0.5,
        color='black',
    )
    .configure_axis(
        grid=False,
    )
    .properties(width=800, height=200)
)
tf.loc[1,'motor cortex'].stack().rename('spikes').loc[lambda x: x > 0]
raster

In [ ]:
from src.extract_smile_data import get_smile_meta

temp = get_smile_meta(smile_data)
temp.groupby(['task','result']).size()
temp

In [ ]:
from src.extract_smile_data import get_trial_events,concat_trial_func_results

events=concat_trial_func_results(get_trial_events, smile_data)
(
    get_trial_events(smile_data[0])
    .reset_index(level='event')
    .set_index('time')
    .resample('10ms')
    .ffill()
    .rename(columns={'event':'state'})
    .squeeze()
)

In [ ]:
from src.extract_smile_data import get_trial_spike_times,bin_spikes,concat_trial_func_results

spike_times = concat_trial_func_results(
    get_trial_spike_times,
    smile_data,
    keep_sorted_only=True,
)
bin_spikes(spike_times)
spike_times

In [ ]:
smile_data[20]['TrialData']['stateTransitions'].T

In [ ]:
pd.DataFrame(smile_data[20]['Parameters']['StateTable']).columns
#pd.DataFrame(smile_data[0]['Parameters']['StateTable'])['stateName']
smile_data[21]['Parameters']['StateTable'][3]['stateName']

In [ ]:
import numpy as np
import altair as alt
from src.extract_smile_data import get_trial_hand_data,concat_trial_func_results

hand_data = concat_trial_func_results(
    get_trial_hand_data,
    smile_data,
    final_sampling_rate=100,
)

def sig_chart(df, **kwargs):
    return (
        alt.Chart(
            df
            .loc[:'0.2s']
            .reset_index()
            .assign(time=lambda x: x['time'].dt.total_seconds())
        )
        .mark_line(
            point='transparent',
            **kwargs
        )
        .encode(
            x='time:Q',
            y=alt.Y('x:Q').scale(zero=False),
        )
    )

(
    # sig_chart(marker_pos_df, color='black')
    # +
    sig_chart(hand_data.loc[1], color='blue')
)

In [ ]:
1/pd.to_timedelta('10ms').total_seconds()

In [ ]:
hand_data

In [ ]:
from src.extract_smile_data import get_spike_waveforms
from sklearn.decomposition import PCA
import altair as alt

waveforms = get_spike_waveforms(smile_data)

channel_waveforms = (
    waveforms
    .groupby('channel')
    .get_group(96)
    .rename(columns=lambda x: str(x))
)

channel_waveform_pca = pd.DataFrame(
    PCA(n_components=2)
    .fit_transform(
        channel_waveforms
    ),
    columns=['PC1','PC2'],
    index=channel_waveforms.index,
)

channel_waveforms = channel_waveforms.assign(**channel_waveform_pca)

sampled_waveforms = (
    channel_waveforms
    .sample(n=500)
    .reset_index()
)

# alt.data_transformers.enable('json')
selection = alt.selection_interval()

scatter = (
    alt.Chart(sampled_waveforms)
    .mark_point()
    .encode(
        x='PC1:Q',
        y='PC2:Q',
        color=alt.condition(
            selection,
            'unit:N',
            alt.value('lightgray'),
        ),
    )
    .add_selection(
        selection
    )
)

waveforms = (
    alt.Chart(sampled_waveforms)
    .mark_line()
    .encode(
        x='frame:Q',
        y='amplitude:Q',
        color=alt.condition(
            selection,
            'unit:N',
            alt.value('lightgray'),
        ),
        detail='snippet_id:O',
    )
    .transform_fold(
        [str(i) for i in range(30)],
        as_=['snippet_frame','amplitude'],
    )
    .transform_calculate(
        frame='toNumber(datum.snippet_frame)',
    )
)
scatter | waveforms

In [ ]:
tf